In [1]:
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import math
from ete3 import Tree
from Bio import AlignIO
from Bio.AlignIO.PhylipIO import RelaxedPhylipWriter
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
import plotly.express as px

In [2]:
def rf_distance_ete(t1, t2):
    rf, max_rf, common_leaves, parts_t1, parts_t2,discard_t1, discart_t2 = t1.robinson_foulds(t2, unrooted_trees = True)
    if max_rf == 0:
        print("?!")
        return 0
    return rf/max_rf


In [3]:
duplicates = pd.read_csv("duplicates.csv")
lang_data = pd.read_parquet("training_data/lang/binary.parquet")

In [31]:
taxa_diff = []
site_diff = []
pattern_diff = []
rfd = []
difficult_diff = []
for i,row in duplicates.iterrows():
    phlorest_name = row["phlorest_name"]
    lexibank_name = row["lexibank_name"]
    print(phlorest_name)
    print(lexibank_name)
    phlorest_align = AlignIO.read("alignments/lang/bin/" + phlorest_name, "phylip-relaxed")
    lexibank_align = AlignIO.read("alignments/lang/bin/" + lexibank_name, "phylip-relaxed")
    phlorest_taxa = set([phlorest_align[i].id for i in range(len(phlorest_align))])
    lexibank_taxa = set([lexibank_align[i].id for i in range(len(lexibank_align))])
    common_taxa = phlorest_taxa.intersection(lexibank_taxa)
    phlorest_taxa_only = phlorest_taxa.difference(common_taxa)
    lexibank_taxa_only = lexibank_taxa.difference(common_taxa)
    if (len(phlorest_taxa_only) != 0 or len(lexibank_taxa_only) != 0):
        print("Different Taxa:")
        print(phlorest_taxa_only)
        print(lexibank_taxa_only)
    taxa_diff.append(len(phlorest_taxa_only) + len(lexibank_taxa_only))

    phlorest_data = lang_data.loc[lang_data["verbose_name"] == row["phlorest_name"]].iloc[0]
    lexibank_data = lang_data.loc[lang_data["verbose_name"] == row["lexibank_name"]].iloc[0]
    print("Sites")
    print(phlorest_data["num_sites"])
    print(lexibank_data["num_sites"])
    site_diff.append(abs(phlorest_data["num_sites"] - lexibank_data["num_sites"]))
    
    print("Patterns")
    print(phlorest_data["num_patterns"])
    print(lexibank_data["num_patterns"])
    pattern_diff.append(abs(phlorest_data["num_patterns"] - lexibank_data["num_patterns"]))
    
    print("RF Distance eval trees")
    t_phlorest = Tree(phlorest_data["newick_eval"])
    t_lexibank = Tree(lexibank_data["newick_eval"])
    rf = rf_distance_ete(t_phlorest, t_lexibank)
    print(rf)
    rfd.append(rf)
    
    print("Difficult")
    print(phlorest_data["difficult"])
    print(lexibank_data["difficult"])
    difficult_diff.append(abs(phlorest_data["difficult"] - lexibank_data["difficult"]))
    print("_______________________________________________________________")
    print("")
    
duplicates["taxa_diff"] = taxa_diff
duplicates["site_diff"] = site_diff
duplicates["pattern_diff"] = pattern_diff
duplicates["rfd_eval"] = rfd
duplicates["difficult_diff"] = difficult_diff
    
    

gray_and_atkinson2003.BIN.cc.phy
dyenindoeuropean.BIN.cc.phy
Different Taxa:
{'Hittite', 'Tocharian_B', 'Tocharian_A'}
{'91', '87', '89', '90', '93', '86', '88', '95', '94', '85', '92'}
Sites
2449
2388
Patterns
1188
2388
RF Distance eval trees
0.19753086419753085
Difficult
0.11816176543209875
0.049344
_______________________________________________________________

koile_et_al2022.BIN.cc.phy
grollemundbantu.BIN.cc.phy
Different Taxa:
set()
{'d308bodo2', 'd304homa1919', 'd308ebodo', 'd20bvamba1919', 'd305nyangali'}
Sites
3859
3853
Patterns
3845
3853
RF Distance eval trees
0.04567307692307692
Difficult
0.4931812000000001
0.5060485974025973
_______________________________________________________________

grollemund_et_al2015.BIN.cc.phy
grollemundbantu.BIN.cc.phy
Sites
3859
3853
Patterns
3859
3853
RF Distance eval trees
0.06413301662707839
Difficult
0.514471
0.5060485974025973
_______________________________________________________________

kitchen_et_al2009.BIN.cc.phy
kitchensemitic.BIN.c

In [36]:
fig = px.histogram(duplicates, x="taxa_diff", nbins=20)
fig.show()


In [29]:
fig = px.histogram(duplicates, x="site_diff", nbins=60)
fig.show()


In [33]:
fig = px.histogram(duplicates, x="pattern_diff", nbins=60)
fig.show()

In [34]:
fig = px.histogram(duplicates, x="difficult_diff", nbins=100)
fig.show()

In [35]:
fig = px.histogram(duplicates, x="rfd_eval", nbins=100)
fig.show()